In [158]:
import spotipy
import webbrowser
import json
import urllib.request
import geocoder
from nltk.corpus import stopwords
import spotipy.util as util
import random

In [159]:
# copy and paste the url into your browser to see what data you are getting back
request = urllib.request.Request("https://quandyfactory.com/insult/json")
response = urllib.request.urlopen(request)

In [160]:
#checking the type of my response
type(response)

http.client.HTTPResponse

In [161]:
# converting this HTTPresponse into its own dictionary
insults = json.loads(response.read())

In [162]:
insults

{'insult': 'Thou art a Gleeking, Shard-borne Vassal.',
 'source': 'The Elizabethan Insult Generator',
 'sourceUrl': 'http://www.museangel.net/insult.html#generator'}

In [163]:
# extracting the keys from the json file that we want to use as our prompt to create spotify playlist
insult = insults['insult']

In [164]:
# I want to go through each word of each title
for word in insult.split(" "):
    print(word)
    # .split(" "): will prevent python from merely printing each character

Thou
art
a
Gleeking,
Shard-borne
Vassal.


In [165]:
list_of_words = insults['insult'].split(" ")

In [166]:
print(list_of_words)
# these are what I want to effectively have as each song of my playlist to spell the insult through an acrostic

['Thou', 'art', 'a', 'Gleeking,', 'Shard-borne', 'Vassal.']


In [167]:
# open file with keys and set the path to your credentials JSON file
credentials = "spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [168]:
# read the keys and assign each to a variable
# A redirect URI, or reply URL, is the location where the authorization server sends the user once the app has been 
# successfully authorized and granted an authorization code or access token.
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [179]:
# scope is requesting special permissions to make playlist from spotify
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-private user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

Enter the URL you were redirected to: https://www.google.com/?code=AQCG2iKy_M6Ofrwa5K9nlv42q6y3OgZHOMQrZnZiQDFdBUbERoIe9ROwngjq0TrRLxQoP4kRNygVdryaIZo_Ort4m-MxOVKDNzWtpuDu52tRMSl5zQDjfcVHa5U52GzSHlh-7XkxFtkAKTJW0RVulbDIvvK7BnzqnqsihmTaRxM1crTwsxtN-QrUloyzh27XQc7tLuBrYqKkt39frD2iedyFqk9uk0I7WMv3q5TACwfXLjLA-mn0HN8VIHgG8C6tXEu5wWRaO-qnhZBcp1gT9AR_PmhKeRfBfIzsTsWJFETGZDbpiZjy1RSljhTUl2Jw


In [180]:
# create spotify object
# this is how we will interact with our api
sp = spotipy.Spotify(auth=token)

In [201]:
songs = []
for search_term in list_of_songs:
    searchResults = sp.search(q="artist:" + search_term, type="track")
    if len(searchResults['tracks']['items']) > 0:
        first_song = searchResults['tracks']['items'][0]
        songs.append(first_song['uri'])

In [202]:
my_playlist = sp.user_playlist_create(user=username, name="insult from an elizabethan", public=False,
                                      description="spotify playlist made with elizabethan insult api")
results = sp.user_playlist_add_tracks(username, my_playlist['id'], songs)
print(results)

{'snapshot_id': 'Myw3NWRiNGQ1NGU5MDQyNjk5Y2VjMzEwY2RmNGZmOTVhYTIwNWEzY2Fk'}


In [203]:
webbrowser.open(my_playlist['external_urls']['spotify'])

True